In [4]:
# Importing the Required Modules

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv

In [5]:
# !pip install python-dotenv

In [2]:
# Function to extract product Title
def get_title(soup):
	try:
		# get the product title as a string
		title = soup.find("span", attrs={"id": "productTitle"}).text.strip()
	
	except AttributeError:
		title = ""
	return title

# Function to extract Product Price
def get_price(soup):
	try:
		# get the product title as a string
		price = soup.find("span", attrs={"class": "a-offscreen"}).text.strip()
	except AttributeError:
		try:
			# If there is some deal price
			price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()
			
		except:
			price = ""
	return price

# Function to extract product rating
def get_rating(soup):
	try:
		# get the product rating as a string
		rating = soup.find("span", attrs={"class": "a-icon-alt"}).text.strip()
	except AttributeError:
		rating = ""
	return rating

# Function to get the number of product reviews by amazon users
def get_review_count(soup):
	try:
		# get the number of product reviews as a string
		review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).text.strip()
	except AttributeError:
		review_count = ""
	return review_count

# Function to extract the availability status of a product
def get_availability(soup):
	try:
		# get the availability status of a product
		availability = soup.find("div", attrs={'id':'availability'})
		availability = availability.find("span").text.strip()
	except AttributeError:
		availability = "Not in Stock"
	return availability

In [16]:
if __name__ == '__main__':

	# The Amazon URL to check
	url = "https://www.amazon.com/s?k=laptop&crid=3S0WZLZ9Y84JK&sprefix=laptop%2Caps%2C592&ref=nb_sb_noss_1"
	
	
	# read my credentials using dotenv
	load_dotenv()
	user_agent = os.getenv("USER_AGENT")
	
	# my header
	header = ({'user-agent': user_agent, 'Accept-Language': 'en-US, em;q=0.5'})

	# HTTP request to the amazon website

	amazon = requests.get(url, headers=header).text

	# soup object containing all HTML data
	soup = BeautifulSoup(amazon, "html.parser")

	# Fetch links from the HTML archor tags
	links = soup.find_all("a", attrs={'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

	# Store the links in an list
	links_list = []

	# Extract the links from the links text
	for link in links:
		links_list.append(link.get('href'))
	
	# create a dictionary for the data

	cols = {"title":[], "price":[], "rating":[], "reviews":[], "availability":[]}

	# extract product detail from all the links in the links_list
	for link in links_list:
		amazon_page = requests.get("https://www.amazon.com" + link, headers=header)
		sub_soup = BeautifulSoup(amazon_page.content, "html.parser")

		# Function calls to display all necessary product information
		cols['title'].append(get_title(sub_soup))
		cols['price'].append(get_price(sub_soup))
		cols['rating'].append(get_rating(sub_soup))
		cols['reviews'].append(get_review_count(sub_soup))
		cols['availability'].append(get_availability(sub_soup))
	
	amazon_data = pd.DataFrame.from_dict(cols)
	amazon_data['title'].replace('',np.nan, inplace=True)
	amazon_data = amazon_data.dropna(subset=['title'])
	amazon_data.to_csv("amazon_data.csv", header=True, index=True)

In [17]:
amazon_data

,title,price,rating,reviews,availability
0,"Lenovo Ideapad 15.6"" HD Laptop, Athlon Silver ...",$399.00,4.8 out of 5 stars,10 ratings,In Stock
1,"Lenovo IdeaPad 1i Laptop 14"" HD Display, Intel...",$219.00,5.0 out of 5 stars,7 ratings,In Stock
2,"SGIN Laptop 15.6 Inch, 4GB DDR4 128GB SSD Wind...",$259.99,4.5 out of 5 stars,"1,377 ratings",In Stock
3,"Lenovo 2022 Newest Ideapad 3 Laptop, 15.6"" HD ...",$379.89,4.4 out of 5 stars,"1,976 ratings",In Stock
4,"2021 HP Stream 14"" HD SVA Laptop Computer, Int...",$289.00,4.3 out of 5 stars,801 ratings,In Stock
5,"Acer Aspire 5 15.6 FHD Notebook Laptop, Intel ...",$829.00,4.4 out of 5 stars,100 ratings,In Stock
6,"Lenovo 15.6"" IdeaPad 1 Laptop, AMD Dual-core P...",$399.00,4.0 out of 5 stars,64 ratings,In Stock
7,"HP 2022 New 15 Laptop, 15.6"" HD LED Display, I...",$479.00,4.3 out of 5 stars,529 ratings,In Stock
8,"HP 2022 Newest Pavilion 15.6"" HD Laptop, Intel...",$469.00,4.3 out of 5 stars,163 ratings,
9,"HP 15 Notebook, 15.6"" HD Screen Laptop, Intel ...",$419.98,4.5 out of 5 stars,219 ratings,In Stock
